# Plan: RANZCR CLiP - Catheter and Line Position Challenge

Objectives:
- Build a strong baseline fast; iterate to medal-level AUC-ROC.
- Use 5-fold CV with robust validation and AMP; generate test predictions and submission.csv.

Data:
- train/: chest X-ray JPGs
- test/: chest X-ray JPGs
- train.csv: image_id + 11 target columns (multilabel).
- sample_submission.csv: required columns and order.
- train_annotations.csv: optional localization info (ignore initially).

Metric:
- Mean macro ROC-AUC across the 11 classes. Track per-class and macro; early-stop/select by macro.

Baseline Modeling Strategy (Phase 1):
- Model: timm tf_efficientnet_b3 (or b3_ns), head: 11 outputs.
- Image size: 512.
- Preproc: Read RGB (grayscale effectively), normalize to ImageNet.
- CLAHE: apply (clip_limit≈2.0, tile_grid_size=(8,8)); include in pipeline (p=0.3–1.0).
- Resize policy: scale longer side to 512, pad shorter side to square; same for train/val/test. Do NOT use RandomResizedCrop.
- Augmentations (safe): HorizontalFlip(0.5), Rotate ±7–10°, ShiftScaleRotate small (shift≤0.05, scale 0.95–1.05), RandomBrightnessContrast(±0.1–0.15), RandomGamma(0.8–1.2), optional light Sharpen/Noise. Avoid elastic/grid/perspective, strong crops, CoarseDropout.
- Loss: BCEWithLogitsLoss with per-class pos_weight = N_neg/N_pos (clipped to [1,5]).
- Optim: AdamW (lr=2e-4–3e-4, wd=1e-4); Scheduler: Cosine with 1-epoch linear warmup.
- Training: 5-fold MultilabelStratifiedKFold (or Group if patient/study ID available). Seed everything.
- Epochs: 6–10 with EarlyStopping(patience=2) on macro AUC; AMP on; EMA optional (decay≈0.9998).
- Batch size: as high as fits (aim 24–32 at 512 with AMP); use grad accumulation if needed.
- TTA: start with 2x (orig + hflip). Average logits across TTAs and folds, then sigmoid.

Phase 2 Improvements (time permitting):
- Add convnext_tiny @512 for diversity and mean-logit ensemble (+0.003–0.006 AUC typical).
- Short finetune best model at 640 px for 2–3 epochs, or train effb4 @640 single model.
- Expand TTA up to 4 variants (add slight rotate/brightness) if stable.

Validation & Logging:
- Use iterative multilabel stratification; log fold composition, seeds.
- Compute per-class AUC and macro AUC per epoch/fold; save best checkpoints by macro.
- Save OOF predictions for analysis; ensure deterministic behavior.

Deliverables:
- submission.csv matching sample_submission.csv format (ids + 11 columns in exact order).

Implementation Priorities (24h):
- Hour 0–2: Data module with CLAHE+resize/pad, MultilabelStratifiedKFold, class counts/pos_weight, macro AUC metric. Verify formats.
- Hour 2–10: Train effb3@512 5 folds with AMP, cosine+warmup, early stop; log thoroughly.
- Hour 10–12: Inference with hflip TTA; fold-average logits; create submission and submit.
- Hour 12–20: Train convnext_tiny@512; ensemble with effb3; resubmit.
- Hour 20–24: Optional 640 finetune or higher-res single model; update ensemble.

Quick wins / Pitfalls:
- Wins: CLAHE, correct pos_weight, iterative stratification, hflip TTA, consistent resize/pad, EMA.
- Pitfalls: wrong metric (micro), RandomResizedCrop, leakage from non-grouped CV, averaging probs instead of logits, inconsistent preprocessing, over-strong augs.

Next Steps:
1) Inspect CSVs (train.csv, sample_submission.csv) to confirm IDs/targets/order.
2) Implement datasets/transforms, pos_weight computation, and CV splitter.
3) Implement baseline training pipeline (EfficientNet-B3 @512) with 5-fold.
4) Generate OOF CV; if strong (>=0.968 CV), proceed to inference + submit.
5) Train convnext_tiny and ensemble if time remains.

In [ ]:
# Setup, inspect CSVs, compute class frequencies and pos_weight
import os, sys, time, json, math, random, gc, glob, platform, subprocess
from pathlib import Path
import numpy as np
import pandas as pd

def pip_install_if_needed(pkg, import_name=None, extra_args=None):
    import importlib
    name = import_name or pkg.split('==')[0]
    try:
        importlib.import_module(name)
        print(f"[pkg] {pkg} already installed")
    except ImportError:
        cmd = [sys.executable, '-m', 'pip', 'install', pkg]
        if extra_args:
            cmd += extra_args
        print('[pip] Installing', pkg)
        subprocess.check_call(cmd)

# Ensure required packages
pip_install_if_needed('timm')
pip_install_if_needed('albumentations')
pip_install_if_needed('iterative-stratification', import_name='iterstrat')

print('[env] Python', platform.python_version())
print('[env] CWD:', os.getcwd())

train_csv_path = 'train.csv'
ss_csv_path = 'sample_submission.csv'
train_dir = Path('train')
test_dir = Path('test')

assert Path(train_csv_path).exists(), 'train.csv not found'
assert Path(ss_csv_path).exists(), 'sample_submission.csv not found'
assert train_dir.exists(), 'train/ dir not found'
assert test_dir.exists(), 'test/ dir not found'

train_df = pd.read_csv(train_csv_path)
ss_df = pd.read_csv(ss_csv_path)
print('[data] train_df shape:', train_df.shape)
print('[data] sample_submission shape:', ss_df.shape)
print('[data] train_df columns:', list(train_df.columns))
print('[data] sample_submission columns:', list(ss_df.columns))

# Identify ID column and target columns using sample_submission format
id_col = ss_df.columns[0]
target_cols = list(ss_df.columns[1:])
print('[data] id_col:', id_col)
print('[data] target_cols (n=%d):' % len(target_cols), target_cols)

# Sanity: ensure train has id_col and targets
missing_cols = [c for c in [id_col] + target_cols if c not in train_df.columns]
assert not missing_cols, f'Missing expected columns in train.csv: {missing_cols}'

# Check a few files existence
n_missing_train = 0
for i, row in train_df.iloc[:50].iterrows():
    img_path = train_dir / f"{row[id_col]}.jpg"
    if not img_path.exists():
        n_missing_train += 1
print(f"[data] Missing train images in first 50: {n_missing_train}")
print(f"[data] Num train images on disk: {len(list(train_dir.glob('*.jpg')))}")
print(f"[data] Num test images on disk: {len(list(test_dir.glob('*.jpg')))}")

# Class frequencies and pos_weight = N_neg/N_pos clipped to [1,5]
y = train_df[target_cols].astype(np.float32)
pos_counts = y.sum(axis=0)
neg_counts = len(y) - pos_counts
pos_weight = (neg_counts / pos_counts.clip(lower=1e-6)).clip(lower=1.0, upper=5.0)
class_stats = pd.DataFrame({
    'pos': pos_counts.astype(int),
    'neg': neg_counts.astype(int),
    'pos_frac': (pos_counts / len(y)).round(6),
    'pos_weight_clipped': pos_weight.round(4)
})
print('[stats] Class stats and pos_weight (clipped to [1,5]):')
print(class_stats)

# Save pos_weight vector for later use
pos_weight_path = 'pos_weight.npy'
np.save(pos_weight_path, pos_weight.values.astype(np.float32))
print('[stats] Saved pos_weight to', pos_weight_path)

# Seed utility
def seed_everything(seed: int = 42):
    import torch
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything(42)
print('[seed] Set to 42')

# Detect GPU
try:
    import torch
    print('[env] Torch', torch.__version__, 'CUDA:', torch.cuda.is_available())
    if torch.cuda.is_available():
        print('[env] GPU:', torch.cuda.get_device_name(0))
except Exception as e:
    print('[env] Torch not available yet or error:', e)

/app/.pip-target/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[pkg] timm already installed


In [13]:
# Dataset, transforms, and CV splitter (Grouped multilabel stratification by PatientID). Sanity-check a batch.
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

def ensure_pkg(pkg, import_name=None):
    import importlib, subprocess, sys
    name = import_name or pkg
    try:
        importlib.import_module(name)
    except ImportError:
        print('[pip] Installing', pkg)
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])

# opencv for CLAHE if not present (headless variant)
ensure_pkg('opencv-python-headless', 'cv2')
ensure_pkg('albumentations')
import albumentations as A
from albumentations.pytorch import ToTensorV2

IMG_SIZE = 640
MEAN = (0.485, 0.456, 0.406)
STD = (0.229, 0.224, 0.225)

def build_train_aug(size=IMG_SIZE, clahe_p=1.0):
    return A.Compose([
        A.CLAHE(clip_limit=2.0, tile_grid_size=(8,8), p=clahe_p),
        A.LongestMaxSize(max_size=size),
        A.PadIfNeeded(min_height=size, min_width=size, border_mode=cv2.BORDER_REFLECT),
        A.HorizontalFlip(p=0.5),
        A.Affine(translate_percent=(0,0.05), scale=(0.95,1.05), rotate=(-7,7), mode=cv2.BORDER_REFLECT, p=0.3),
        A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.2),
        A.RandomGamma(gamma_limit=(80,120), p=0.2),
        A.Normalize(mean=MEAN, std=STD),
        ToTensorV2()
    ])

def build_valid_aug(size=IMG_SIZE, clahe_p=1.0):
    return A.Compose([
        A.CLAHE(clip_limit=2.0, tile_grid_size=(8,8), p=clahe_p),
        A.LongestMaxSize(max_size=size),
        A.PadIfNeeded(min_height=size, min_width=size, border_mode=cv2.BORDER_REFLECT),
        A.Normalize(mean=MEAN, std=STD),
        ToTensorV2()
    ])

class RanzcrDataset(Dataset):
    def __init__(self, df: pd.DataFrame, img_dir: Path, id_col: str, target_cols: list[str], transforms=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.id_col = id_col
        self.target_cols = target_cols
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.img_dir / f"{row[self.id_col]}.jpg"
        image = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
        if image is None:
            raise FileNotFoundError(f"Image not found: {img_path}")
        image = np.stack([image, image, image], axis=-1)
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        targets = row[self.target_cols].values.astype(np.float32)
        return image, torch.from_numpy(targets)

def make_grouped_ml_folds(df: pd.DataFrame, target_cols: list[str], group_col='PatientID', n_splits=5, seed=42):
    grp = df.groupby(group_col)[target_cols].max().reset_index()
    mskf = MultilabelStratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    grp['fold'] = -1
    for f, (_, vidx) in enumerate(mskf.split(grp, grp[target_cols].values)):
        grp.loc[grp.index[vidx], 'fold'] = f
    folds_df = df.merge(grp[[group_col, 'fold']], on=group_col, how='left')
    folds_df['fold'] = folds_df['fold'].astype(int)
    return folds_df

# Create grouped multilabel folds and save
folds_path = 'train_folds.csv'
folds_df = make_grouped_ml_folds(train_df, target_cols, group_col='PatientID', n_splits=5, seed=42)
folds_df.to_csv(folds_path, index=False)
print('[cv] Saved grouped multilabel folds to', folds_path)

# Log fold sizes and label balance snapshot
for f in range(5):
    cnt = (folds_df['fold'] == f).sum()
    pos_frac = folds_df.loc[folds_df['fold']==f, target_cols].mean().round(4)
    print(f"[cv] fold {f}: n={cnt}, pos_fracs(sample):", dict(pos_frac.head(5)))

# Sanity check a dataloader batch
fold = 0
train_idx = folds_df.index[folds_df['fold'] != fold].tolist()
valid_idx = folds_df.index[folds_df['fold'] == fold].tolist()
train_ds = RanzcrDataset(folds_df.iloc[train_idx], train_dir, id_col, target_cols, transforms=build_train_aug())
valid_ds = RanzcrDataset(folds_df.iloc[valid_idx], train_dir, id_col, target_cols, transforms=build_valid_aug())
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2)
valid_loader = DataLoader(valid_ds, batch_size=8, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2)
t0 = time.time()
xb, yb = next(iter(train_loader))
print('[loader] one batch shapes:', xb.shape, yb.shape, 'elapsed', round(time.time()-t0,2), 's')
print('[loader] xb stats: mean', xb.float().mean().item(), 'std', xb.float().std().item())

[cv] Saved grouped multilabel folds to train_folds.csv
[cv] fold 0: n=5284, pos_fracs(sample): {'ETT - Abnormal': 0.003, 'ETT - Borderline': 0.0367, 'ETT - Normal': 0.204, 'NGT - Abnormal': 0.0083, 'NGT - Borderline': 0.0155}
[cv] fold 1: n=5402, pos_fracs(sample): {'ETT - Abnormal': 0.0026, 'ETT - Borderline': 0.0446, 'ETT - Normal': 0.2599, 'NGT - Abnormal': 0.007, 'NGT - Borderline': 0.0193}
[cv] fold 2: n=5140, pos_fracs(sample): {'ETT - Abnormal': 0.0029, 'ETT - Borderline': 0.0372, 'ETT - Normal': 0.2307, 'NGT - Abnormal': 0.0154, 'NGT - Borderline': 0.0152}
[cv] fold 3: n=5388, pos_fracs(sample): {'ETT - Abnormal': 0.003, 'ETT - Borderline': 0.0395, 'ETT - Normal': 0.2359, 'NGT - Abnormal': 0.0076, 'NGT - Borderline': 0.0239}
[cv] fold 4: n=5860, pos_fracs(sample): {'ETT - Abnormal': 0.0026, 'ETT - Borderline': 0.0321, 'ETT - Normal': 0.2669, 'NGT - Abnormal': 0.0089, 'NGT - Borderline': 0.016}


/app/.pip-target/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


[loader] one batch shapes: torch.Size([8, 3, 640, 640]) torch.Size([8, 9]) elapsed 1.41 s
[loader] xb stats: mean 0.1559348702430725 std 1.1031512022018433


In [15]:
# Model, training loop with 5-fold CV, macro AUC metric, checkpoints (stabilized: cosine per-step, EMA off, sigmoid CV alignment)
import os, time, math, gc
from sklearn.metrics import roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
from timm.scheduler import CosineLRScheduler
import timm
import numpy as np

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
EPOCHS = 12
BATCH_SIZE = 8  # per-GPU
ACCUM_STEPS = 2  # effective batch 16
LR = 1.2e-4  # base LR for cosine
WD = 1e-4
EMA_USE = False  # disable EMA per stability guidance
EARLY_PATIENCE = 4
LS = 0.05  # label smoothing for BCE (manual)

os.makedirs('chkpts', exist_ok=True)
# ensure writable cache for HF/timm
os.makedirs('hf_cache', exist_ok=True)
os.environ['HF_HOME'] = os.path.abspath('hf_cache')
os.environ['HUGGINGFACE_HUB_CACHE'] = os.path.abspath('hf_cache')
os.environ['XDG_CACHE_HOME'] = os.path.abspath('hf_cache')
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

def build_model(num_classes: int):
    model = timm.create_model(
        'tf_efficientnet_b4_ns',
        pretrained=True,
        num_classes=num_classes,
        in_chans=3,
        cache_dir=os.path.abspath('hf_cache')
    )
    # Sanity: classifier properties
    clf = model.get_classifier()
    try:
        outf = getattr(clf, 'out_features', None)
        wmean = clf.weight.mean().item() if hasattr(clf, 'weight') else None
        print('[model] pretrained=True; num_outputs', outf, 'fc_mean', wmean)
    except Exception as e:
        print('[model] classifier introspection error:', e)
    return model

def get_loaders_for_fold(fold: int, img_size=IMG_SIZE):
    train_idx = folds_df.index[folds_df['fold'] != fold].tolist()
    valid_idx = folds_df.index[folds_df['fold'] == fold].tolist()
    train_ds = RanzcrDataset(folds_df.iloc[train_idx], train_dir, id_col, target_cols, transforms=build_train_aug(img_size, clahe_p=1.0))
    valid_ds = RanzcrDataset(folds_df.iloc[valid_idx], train_dir, id_col, target_cols, transforms=build_valid_aug(img_size, clahe_p=1.0))
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2)
    valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2)
    return train_loader, valid_loader, valid_idx

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def evaluate_macro_auc(y_true_np, y_pred_np):
    aucs = []
    for i in range(y_true_np.shape[1]):
        y_true_c = y_true_np[:, i]
        y_pred_c = y_pred_np[:, i]
        try:
            aucs.append(roc_auc_score(y_true_c, y_pred_c))
        except Exception:
            continue
    macro = float(np.mean(aucs)) if len(aucs)>0 else 0.0
    return macro, aucs

def train_one_fold(fold: int):
    print(f"\n===== Fold {fold} training start =====")
    gc.collect()
    if DEVICE=='cuda':
        torch.cuda.empty_cache()
    print('[debug] target_cols:', target_cols)
    train_loader, valid_loader, valid_idx = get_loaders_for_fold(fold, IMG_SIZE)
    model = build_model(num_classes=len(target_cols)).to(DEVICE)
    model = model.to(memory_format=torch.channels_last)

    pos_w = torch.from_numpy(np.load('pos_weight.npy')).to(DEVICE)
    pos_w = pos_w.clamp(min=1.0, max=5.0)
    print('[DEBUG] Initial pos_weight:', pos_w.detach().cpu().numpy().tolist())
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_w)
    optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)

    # Per-step cosine scheduler with 1-epoch warmup
    steps_per_epoch = max(1, len(train_loader) // ACCUM_STEPS)
    total_steps = EPOCHS * steps_per_epoch
    warmup_steps = steps_per_epoch
    lr_sched = CosineLRScheduler(
        optimizer,
        t_initial=total_steps,
        lr_min=max(1e-6, LR * 0.05),
        warmup_t=warmup_steps,
        warmup_lr_init=LR / 10.0,
        warmup_prefix=True,
        t_in_epochs=False
    )

    scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=='cuda'))
    best_auc = -1.0
    best_path = f"chkpts/effb4ns_fold{fold}.pt"
    best_val_logits = None
    no_improve = 0
    t_fold = time.time()
    global_step = 0
    for epoch in range(EPOCHS):
        model.train()
        t0 = time.time()
        running_loss = 0.0
        optimizer.zero_grad(set_to_none=True)
        for it, (xb, yb) in enumerate(train_loader):
            if it == 0:
                try:
                    print('[debug] first-batch targets sum per class:', yb.sum(0).tolist())
                except Exception:
                    pass
            xb = xb.to(DEVICE, non_blocking=True).to(memory_format=torch.channels_last)
            yb = yb.to(DEVICE, non_blocking=True)
            # manual label smoothing to keep targets in [LS, 1-LS]
            yb_s = yb * (1.0 - 2.0 * LS) + LS
            with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
                logits = model(xb)
                loss = criterion(logits, yb_s)
            scaler.scale(loss / ACCUM_STEPS).backward()
            if (it + 1) % ACCUM_STEPS == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad(set_to_none=True)
                lr_sched.step_update(global_step)
                global_step += 1
            running_loss += loss.item()
            if (it+1) % 50 == 0:
                elapsed = time.time()-t0
                cur_lr = optimizer.param_groups[0]['lr']
                print(f"[fold {fold}] epoch {epoch+1}/{EPOCHS} it {it+1}/{len(train_loader)} loss {running_loss/(it+1):.4f} lr {cur_lr:.2e} elapsed {elapsed:.1f}s")
        # validate (sigmoid-only, align with inference)
        model.eval()
        val_logits = []
        val_targets = []
        with torch.no_grad():
            for xb, yb in valid_loader:
                xb = xb.to(DEVICE, non_blocking=True).to(memory_format=torch.channels_last)
                logits = model(xb)
                val_logits.append(logits.detach().cpu().numpy())
                val_targets.append(yb.numpy())
        val_logits = np.concatenate(val_logits, axis=0)
        val_targets = np.concatenate(val_targets, axis=0)
        macro_auc, aucs = evaluate_macro_auc(val_targets, sigmoid(val_logits))
        print(f"[fold {fold}] epoch {epoch+1} val macro AUC: {macro_auc:.5f}")
        for cls_name, a in zip(target_cols, aucs):
            try:
                print(f"  {cls_name}: {a:.4f}")
            except Exception:
                pass
        if macro_auc > best_auc:
            best_auc = macro_auc
            no_improve = 0
            torch.save({'model': model.state_dict(), 'auc': best_auc}, best_path)
            print(f"[fold {fold}] saved best to {best_path}")
            best_val_logits = val_logits.copy()
        else:
            no_improve += 1
        print(f"[fold {fold}] epoch {epoch+1} train_loss {running_loss/max(1,len(train_loader)):.4f} time {time.time()-t0:.1f}s")
        if no_improve >= EARLY_PATIENCE:
            print(f"[fold {fold}] Early stopping at epoch {epoch+1}")
            break
    print(f"[fold {fold}] best macro AUC: {best_auc:.5f} total time {time.time()-t_fold:.1f}s")
    return best_auc, valid_idx, best_val_logits

# Orchestrate training: run all 5 folds
oof_logits_all = np.full((len(folds_df), len(target_cols)), np.nan, np.float32)
fold_aucs = []
for f in range(5):
    start = time.time()
    aucf, valid_idx, val_logits_best = train_one_fold(f)
    fold_aucs.append(aucf)
    if val_logits_best is not None:
        oof_logits_all[valid_idx] = val_logits_best
    print(f"[cv] Fold {f} best AUC: {aucf:.5f} elapsed {time.time()-start:.1f}s")

# Compute OOF macro AUC (where filled) for sanity
mask = ~np.isnan(oof_logits_all).any(axis=1)
if mask.sum() > 0:
    y_true_all = folds_df.loc[mask, target_cols].values.astype(np.float32)
    y_pred_all = sigmoid(oof_logits_all[mask])
    oof_macro, _ = evaluate_macro_auc(y_true_all, y_pred_all)
    print(f"[cv] OOF macro AUC (filled folds): {oof_macro:.5f} covering {mask.sum()} samples")
np.save('oof_logits_all.npy', oof_logits_all)
print('[cv] AUCs per fold:', fold_aucs, 'mean:', float(np.mean(fold_aucs)))


===== Fold 0 training start =====
[debug] target_cols: ['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline', 'NGT - Incompletely Imaged', 'NGT - Normal', 'CVC - Abnormal', 'CVC - Borderline']


[model] pretrained=True; num_outputs 9 fc_mean -0.000944784376770258
[DEBUG] Initial pos_weight: [5.0, 5.0, 3.163309335708618, 5.0, 5.0, 5.0, 5.0, 5.0, 2.5474319458007812]


/tmp/ipykernel_1774/544503444.py:104: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=='cuda'))


[debug] first-batch targets sum per class: [0.0, 0.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 3.0]


/tmp/ipykernel_1774/544503444.py:126: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):


...

[fold 4] epoch 7/12 it 1200/2652 loss 0.6026 lr 7.12e-05 elapsed 415.0s


[fold 4] epoch 7/12 it 1250/2652 loss 0.6027 lr 7.09e-05 elapsed 432.3s


[fold 4] epoch 7/12 it 1300/2652 loss 0.6027 lr 7.06e-05 elapsed 449.5s


[fold 4] epoch 7/12 it 1350/2652 loss 0.6029 lr 7.03e-05 elapsed 466.8s


[fold 4] epoch 7/12 it 1400/2652 loss 0.6030 lr 7.00e-05 elapsed 484.0s


In [17]:
# Inference/TTA: load available fold checkpoints, predict test, average logits, apply group-wise softmax, write submission.csv
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

class RanzcrTestDataset(Dataset):
    def __init__(self, df: pd.DataFrame, img_dir: Path, id_col: str, transforms=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.id_col = id_col
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.img_dir / f"{row[self.id_col]}.jpg"
        image = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
        if image is None:
            raise FileNotFoundError(f"Image not found: {img_path}")
        image = np.stack([image, image, image], axis=-1)
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        return image, row[self.id_col]

def load_model_for_infer(ckpt_path: str, num_classes: int):
    # Select architecture based on checkpoint prefix
    model_name = 'tf_efficientnet_b4_ns' if 'effb4ns' in ckpt_path else 'tf_efficientnet_b3'
    model = timm.create_model(
        model_name,
        pretrained=True,
        num_classes=num_classes,
        in_chans=3,
        cache_dir=os.path.abspath('hf_cache')
    ).to(DEVICE)
    state = torch.load(ckpt_path, map_location='cpu')
    model.load_state_dict(state['model'], strict=True)
    model.eval()
    return model

def apply_groupwise_softmax(mean_logits: np.ndarray) -> np.ndarray:
    # mean_logits shape: [N, C] with C=len(target_cols)=9
    out = mean_logits.copy()
    # Indices per sample_submission order:
    # 0-2: ETT [Abnormal, Borderline, Normal], 3-6: NGT [Abnormal, Borderline, Incompletely Imaged, Normal], 7-8: CVC [Abnormal, Borderline]
    ett_idx = [0,1,2]
    ngt_idx = [3,4,5,6]
    cvc_idx = [7,8]
    # softmax for ETT and NGT groups on logits
    def softmax(x, axis=-1):
        x_max = np.max(x, axis=axis, keepdims=True)
        e = np.exp(x - x_max)
        return e / np.sum(e, axis=axis, keepdims=True)
    out[:, ett_idx] = softmax(out[:, ett_idx], axis=1)
    out[:, ngt_idx] = softmax(out[:, ngt_idx], axis=1)
    # for CVC group keep sigmoid over 2 independent labels
    out[:, cvc_idx] = 1.0 / (1.0 + np.exp(-out[:, cvc_idx]))
    return out

def predict_test_tta(fold_ckpts, tta_hflip=True, batch_size=BATCH_SIZE):
    test_df = ss_df[[id_col]].copy()
    test_ds = RanzcrTestDataset(test_df, test_dir, id_col, transforms=build_valid_aug(IMG_SIZE, clahe_p=1.0))
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
    all_logits = []
    used_folds = []
    if DEVICE=='cuda':
        torch.cuda.empty_cache()
    for f, ckpt in enumerate(fold_ckpts):
        if not os.path.exists(ckpt):
            print(f"[infer] skip fold {f}, ckpt not found: {ckpt}")
            continue
        print(f"[infer] loading {ckpt}")
        model = load_model_for_infer(ckpt, num_classes=len(target_cols))
        model.to(DEVICE)
        fold_logits = []
        with torch.no_grad():
            for xb, _ids in test_loader:
                xb = xb.to(DEVICE, non_blocking=True)
                with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
                    logits = model(xb)
                    if tta_hflip:
                        xb_flip = torch.flip(xb, dims=[3])
                        logits_flip = model(xb_flip)
                        logits = (logits + logits_flip) / 2.0
                fold_logits.append(logits.detach().cpu().numpy())
        fold_logits = np.concatenate(fold_logits, axis=0)
        all_logits.append(fold_logits)
        used_folds.append(f)
    if not all_logits:
        raise RuntimeError('No checkpoints found for inference.')
    mean_logits = np.mean(all_logits, axis=0)
    # Apply group-wise post-processing: softmax for ETT and NGT groups, sigmoid for CVC
    probs = apply_groupwise_softmax(mean_logits)
    sub = pd.DataFrame(probs, columns=target_cols)
    sub.insert(0, id_col, ss_df[id_col].values)
    return sub, used_folds

# Run inference using any finished fold checkpoints
fold_ckpts = [f"chkpts/effb3_fold{f}.pt" for f in range(5)]
try:
    submission_df, used_folds = predict_test_tta(fold_ckpts, tta_hflip=True, batch_size=16)
    print('[infer] used folds:', used_folds)
    submission_df.to_csv('submission.csv', index=False)
    print('[infer] saved submission.csv with shape', submission_df.shape)
except Exception as e:
    print('[infer] inference not ready:', e)

[infer] loading chkpts/effb3_fold0.pt


/tmp/ipykernel_1774/2673439433.py:80: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):


In [12]:
# Validate post-processing on OOF and regenerate submission with sigmoid-only probabilities
import numpy as np
from sklearn.metrics import roc_auc_score

def sigmoid_np(x):
    return 1.0 / (1.0 + np.exp(-x))

def softmax_np(x, axis=-1):
    x_max = np.max(x, axis=axis, keepdims=True)
    e = np.exp(x - x_max)
    return e / np.sum(e, axis=axis, keepdims=True)

def apply_groupwise_softmax_logits(logits: np.ndarray) -> np.ndarray:
    out = logits.copy()
    ett_idx = [0,1,2]
    ngt_idx = [3,4,5,6]
    cvc_idx = [7,8]
    out[:, ett_idx] = softmax_np(out[:, ett_idx], axis=1)
    out[:, ngt_idx] = softmax_np(out[:, ngt_idx], axis=1)
    out[:, cvc_idx] = sigmoid_np(out[:, cvc_idx])
    return out

def macro_auc_from_probs(y_true: np.ndarray, probs: np.ndarray) -> float:
    aucs = []
    for i in range(y_true.shape[1]):
        try:
            aucs.append(roc_auc_score(y_true[:, i], probs[:, i]))
        except Exception:
            pass
    return float(np.mean(aucs)) if len(aucs)>0 else 0.0

# Load OOF logits and targets
oof_logits = np.load('oof_logits_all.npy')
mask = ~np.isnan(oof_logits).any(axis=1)
y_true = folds_df.loc[mask, target_cols].values.astype(np.float32)
logits_filled = oof_logits[mask]

# OOF AUC with sigmoid-only
probs_sig = sigmoid_np(logits_filled)
oof_auc_sig = macro_auc_from_probs(y_true, probs_sig)
print('[oof] Macro AUC (sigmoid-only):', f'{oof_auc_sig:.5f}')

# OOF AUC with group-wise post-processing (ETT/NGT softmax on logits, CVC sigmoid)
probs_group = apply_groupwise_softmax_logits(logits_filled)
oof_auc_group = macro_auc_from_probs(y_true, probs_group)
print('[oof] Macro AUC (groupwise softmax + CVC sigmoid):', f'{oof_auc_group:.5f}')

# Re-run test inference to produce sigmoid-only submission.csv
def predict_test_logits(fold_ckpts, tta_hflip=True, batch_size=BATCH_SIZE):
    test_df = ss_df[[id_col]].copy()
    test_ds = RanzcrTestDataset(test_df, test_dir, id_col, transforms=build_valid_aug(IMG_SIZE, clahe_p=1.0))
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
    all_logits = []
    used = []
    if DEVICE=='cuda':
        torch.cuda.empty_cache()
    for f, ckpt in enumerate(fold_ckpts):
        if not os.path.exists(ckpt):
            print(f"[infer2] skip fold {f}, missing {ckpt}")
            continue
        print(f"[infer2] loading {ckpt}")
        model = load_model_for_infer(ckpt, num_classes=len(target_cols)).to(DEVICE)
        model.eval()
        fold_logits = []
        with torch.no_grad():
            for xb, _ids in test_loader:
                xb = xb.to(DEVICE, non_blocking=True)
                with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
                    logits = model(xb)
                    if tta_hflip:
                        xb_flip = torch.flip(xb, dims=[3])
                        logits_flip = model(xb_flip)
                        logits = (logits + logits_flip) / 2.0
                fold_logits.append(logits.detach().cpu().numpy())
        all_logits.append(np.concatenate(fold_logits, axis=0))
        used.append(f)
    assert len(all_logits)>0, 'No logits collected'
    mean_logits = np.mean(all_logits, axis=0)
    return mean_logits, used

fold_ckpts = [f"chkpts/effb3_fold{f}.pt" for f in range(5)]
mean_logits_test, used = predict_test_logits(fold_ckpts, tta_hflip=True, batch_size=16)
print('[infer2] used folds:', used)

# Sigmoid-only probabilities for submission
probs_test_sig = sigmoid_np(mean_logits_test)
sub_sig = pd.DataFrame(probs_test_sig, columns=target_cols)
sub_sig.insert(0, id_col, ss_df[id_col].values)
sub_sig.to_csv('submission.csv', index=False)
print('[infer2] Wrote sigmoid-only submission.csv with shape', sub_sig.shape)

[oof] Macro AUC (sigmoid-only): 0.88729
[oof] Macro AUC (groupwise softmax + CVC sigmoid): 0.58707
[infer2] loading chkpts/effb3_fold0.pt


[model] pretrained=True; num_outputs 9 fc_mean -0.00042216299334540963


/tmp/ipykernel_1774/1325318391.py:68: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):


[infer2] loading chkpts/effb3_fold1.pt


[model] pretrained=True; num_outputs 9 fc_mean 0.0009805666049942374


...

[model] pretrained=True; num_outputs 9 fc_mean -0.0028276173397898674


[infer2] loading chkpts/effb3_fold4.pt


[model] pretrained=True; num_outputs 9 fc_mean 0.003661656752228737


[infer2] used folds: [0, 1, 2, 3, 4]
[infer2] Wrote sigmoid-only submission.csv with shape (3009, 10)


/tmp/ipykernel_1774/1325318391.py:6: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))


In [18]:
# Inference: sigmoid-only for b4_ns folds; optional 0.7/0.3 ensemble with b3 (logits avg), hflip TTA
import os, numpy as np, pandas as pd, torch
from torch.utils.data import DataLoader

def get_available_ckpts(prefix: str, max_folds: int = 5):
    paths = []
    for f in range(max_folds):
        p = f"chkpts/{prefix}_fold{f}.pt"
        if os.path.exists(p):
            paths.append(p)
    return paths

def predict_test_logits_from_ckpts(ckpt_paths, tta_hflip=True, batch_size=16):
    assert len(ckpt_paths) > 0, 'No checkpoints provided'
    test_df = ss_df[[id_col]].copy()
    test_ds = RanzcrTestDataset(test_df, test_dir, id_col, transforms=build_valid_aug(IMG_SIZE, clahe_p=1.0))
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
    all_logits = []
    if DEVICE=='cuda':
        torch.cuda.empty_cache()
    for ckpt in ckpt_paths:
        print(f"[infer] loading {ckpt}")
        model = load_model_for_infer(ckpt, num_classes=len(target_cols)).to(DEVICE)
        model.eval()
        fold_logits = []
        with torch.no_grad():
            for xb, _ids in test_loader:
                xb = xb.to(DEVICE, non_blocking=True)
                with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
                    logits = model(xb)
                    if tta_hflip:
                        xb_flip = torch.flip(xb, dims=[3])
                        logits_flip = model(xb_flip)
                        logits = (logits + logits_flip) / 2.0
                fold_logits.append(logits.detach().cpu().numpy())
        all_logits.append(np.concatenate(fold_logits, axis=0))
    mean_logits = np.mean(all_logits, axis=0)
    return mean_logits

def sigmoid_np(x):
    return 1.0 / (1.0 + np.exp(-x))

def write_submission_from_logits(mean_logits: np.ndarray, out_path: str):
    probs = sigmoid_np(mean_logits)
    sub = pd.DataFrame(probs, columns=target_cols)
    sub.insert(0, id_col, ss_df[id_col].values)
    sub.to_csv(out_path, index=False)
    print('[infer] wrote', out_path, 'shape', sub.shape)

# Prepare inference (do NOT run during training to avoid OOM; run after Cell 3 completes):
if True:
    b4_ckpts = get_available_ckpts('effb4ns')
    print('[infer] effb4ns ckpts:', b4_ckpts)
    assert len(b4_ckpts) > 0, 'No effb4ns checkpoints found'
    logits_b4 = predict_test_logits_from_ckpts(b4_ckpts, tta_hflip=True, batch_size=16)
    write_submission_from_logits(logits_b4, 'submission_b4ns_sigmoid.csv')
    # Optional ensemble with b3
    b3_ckpts = get_available_ckpts('effb3')
    if len(b3_ckpts) > 0:
        logits_b3 = predict_test_logits_from_ckpts(b3_ckpts, tta_hflip=True, batch_size=16)
        blend_logits = 0.7 * logits_b4 + 0.3 * logits_b3
        write_submission_from_logits(blend_logits, 'submission_ens7030_sigmoid.csv')
        os.replace('submission_ens7030_sigmoid.csv', 'submission.csv')
        print('[infer] Final submission.csv written from 70/30 ensemble')
    else:
        os.replace('submission_b4ns_sigmoid.csv', 'submission.csv')
        print('[infer] Final submission.csv written from b4ns only')

[infer] effb4ns ckpts: ['chkpts/effb4ns_fold0.pt', 'chkpts/effb4ns_fold1.pt', 'chkpts/effb4ns_fold2.pt', 'chkpts/effb4ns_fold3.pt', 'chkpts/effb4ns_fold4.pt']
[infer] loading chkpts/effb4ns_fold0.pt


/app/.pip-target/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(


/tmp/ipykernel_1774/3136468299.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):


[infer] loading chkpts/effb4ns_fold1.pt


[infer] loading chkpts/effb4ns_fold2.pt


...

[infer] loading chkpts/effb3_fold1.pt


[infer] loading chkpts/effb3_fold2.pt


[infer] loading chkpts/effb3_fold3.pt


[infer] loading chkpts/effb3_fold4.pt


[infer] wrote submission_ens7030_sigmoid.csv shape (3009, 10)
[infer] Final submission.csv written from 70/30 ensemble


In [19]:
# Step 1: Build b3 OOF logits, tune blend with b4 OOF, generate improved submission (sigmoid-only)
import numpy as np, pandas as pd, os, time
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader
import torch

def sigmoid_np(x):
    return 1.0 / (1.0 + np.exp(-x))

def macro_auc_from_probs(y_true: np.ndarray, probs: np.ndarray) -> float:
    aucs = []
    for i in range(y_true.shape[1]):
        try:
            aucs.append(roc_auc_score(y_true[:, i], probs[:, i]))
        except Exception:
            pass
    return float(np.mean(aucs)) if len(aucs)>0 else 0.0

def get_val_loader_for_fold(fold: int, size: int):
    valid_idx = folds_df.index[folds_df['fold'] == fold].tolist()
    valid_ds = RanzcrDataset(folds_df.iloc[valid_idx], train_dir, id_col, target_cols, transforms=build_valid_aug(size, clahe_p=1.0))
    valid_loader = DataLoader(valid_ds, batch_size=16, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
    return valid_loader, valid_idx

def collect_oof_logits_from_ckpts(ckpt_prefix: str, size: int) -> np.ndarray:
    oof_logits = np.full((len(folds_df), len(target_cols)), np.nan, np.float32)
    if DEVICE=='cuda':
        torch.cuda.empty_cache()
    for f in range(5):
        ckpt = f"chkpts/{ckpt_prefix}_fold{f}.pt"
        if not os.path.exists(ckpt):
            print(f"[oof] missing ckpt for fold {f}: {ckpt}")
            continue
        print(f"[oof] fold {f} loading {ckpt} @size {size}")
        model = load_model_for_infer(ckpt, num_classes=len(target_cols)).to(DEVICE)
        model.eval()
        val_loader, vidx = get_val_loader_for_fold(f, size)
        logits_col = []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(DEVICE, non_blocking=True)
                with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
                    logits = model(xb)
                logits_col.append(logits.detach().cpu().numpy())
        val_logits = np.concatenate(logits_col, axis=0)
        oof_logits[vidx] = val_logits
    return oof_logits

# Load b4 OOF logits (640) from training run
oof_b4_path = 'oof_logits_all.npy'
assert os.path.exists(oof_b4_path), 'Missing b4 OOF logits file'
oof_b4 = np.load(oof_b4_path)
mask_b4 = ~np.isnan(oof_b4).any(axis=1)
y_true = folds_df.loc[mask_b4, target_cols].values.astype(np.float32)
print('[oof] b4 OOF coverage:', mask_b4.sum(), '/', len(mask_b4))

# Build b3 OOF logits (use 512 to match b3 training)
b3_oof_path = 'oof_logits_b3.npy'
if os.path.exists(b3_oof_path):
    oof_b3 = np.load(b3_oof_path)
    print('[oof] Loaded cached b3 OOF logits')
else:
    t0 = time.time()
    oof_b3 = collect_oof_logits_from_ckpts('effb3', size=512)
    np.save(b3_oof_path, oof_b3)
    print('[oof] Saved b3 OOF logits to', b3_oof_path, 'time', round(time.time()-t0,1), 's')

# Align masks
mask_b3 = ~np.isnan(oof_b3).any(axis=1)
mask = mask_b3 & mask_b4
y_true_aln = folds_df.loc[mask, target_cols].values.astype(np.float32)
b4_logits_aln = oof_b4[mask]
b3_logits_aln = oof_b3[mask]

# Baseline AUCs
auc_b4 = macro_auc_from_probs(y_true_aln, sigmoid_np(b4_logits_aln))
auc_b3 = macro_auc_from_probs(y_true_aln, sigmoid_np(b3_logits_aln))
print(f"[oof] AUC b4(sigmoid): {auc_b4:.5f}  b3(sigmoid): {auc_b3:.5f}")

# Grid search blend weight w for mean-logits: sigmoid(w*b4 + (1-w)*b3)
weights = np.linspace(0.0, 1.0, 21)
best_auc = -1.0
best_w = None
for w in weights:
    blend_logits = w * b4_logits_aln + (1.0 - w) * b3_logits_aln
    auc = macro_auc_from_probs(y_true_aln, sigmoid_np(blend_logits))
    if auc > best_auc:
        best_auc = auc
        best_w = float(w)
print(f"[oof] Mean-logits blend best w={best_w:.2f} AUC={best_auc:.5f}")

# Rank-averaging per class (on logits ranks), grid over w
def rank_normalize(mat: np.ndarray) -> np.ndarray:
    # Convert each column to ranks in [0,1]
    n = mat.shape[0]
    ranks = np.zeros_like(mat, dtype=np.float32)
    for j in range(mat.shape[1]):
        order = np.argsort(mat[:, j])
        ranks[order, j] = np.arange(n, dtype=np.float32)
    return ranks / (n - 1 + 1e-9)

b4_ranks = rank_normalize(b4_logits_aln)
b3_ranks = rank_normalize(b3_logits_aln)
best_auc_rank = -1.0
best_w_rank = None
for w in weights:
    blend_rank = w * b4_ranks + (1.0 - w) * b3_ranks
    auc = macro_auc_from_probs(y_true_aln, blend_rank)
    if auc > best_auc_rank:
        best_auc_rank = auc
        best_w_rank = float(w)
print(f"[oof] Rank-avg best w={best_w_rank:.2f} AUC={best_auc_rank:.5f}")

# Choose method
use_rank = best_auc_rank > best_auc
print('[oof] Chosen method:', 'rank-avg' if use_rank else 'mean-logits')
final_w = best_w_rank if use_rank else best_w

# Compute test logits for b4@640 and b3@512, with hflip TTA, then blend and write submission.csv
def predict_test_logits_size(ckpt_prefix: str, size: int, batch_size: int = 16, tta_hflip: bool = True) -> np.ndarray:
    test_df = ss_df[[id_col]].copy()
    test_ds = RanzcrTestDataset(test_df, test_dir, id_col, transforms=build_valid_aug(size, clahe_p=1.0))
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
    all_fold_logits = []
    if DEVICE=='cuda':
        torch.cuda.empty_cache()
    for f in range(5):
        ckpt = f"chkpts/{ckpt_prefix}_fold{f}.pt"
        if not os.path.exists(ckpt):
            print(f"[test] missing {ckpt}, skipping")
            continue
        print(f"[test] loading {ckpt} @size {size}")
        model = load_model_for_infer(ckpt, num_classes=len(target_cols)).to(DEVICE)
        model.eval()
        fold_logits = []
        with torch.no_grad():
            for xb, _ids in test_loader:
                xb = xb.to(DEVICE, non_blocking=True)
                with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
                    logits = model(xb)
                    if tta_hflip:
                        xb_flip = torch.flip(xb, dims=[3])
                        logits_flip = model(xb_flip)
                        logits = (logits + logits_flip) / 2.0
                fold_logits.append(logits.detach().cpu().numpy())
        all_fold_logits.append(np.concatenate(fold_logits, axis=0))
    assert len(all_fold_logits)>0, 'No test logits collected'
    return np.mean(all_fold_logits, axis=0)

t_test = time.time()
logits_b4_test = predict_test_logits_size('effb4ns', size=640, batch_size=16, tta_hflip=True)
logits_b3_test = predict_test_logits_size('effb3', size=512, batch_size=16, tta_hflip=True)
np.save('test_logits_b4_640.npy', logits_b4_test)
np.save('test_logits_b3_512.npy', logits_b3_test)
print('[test] saved test logits b4/b3, time', round(time.time()-t_test,1), 's')

if use_rank:
    # rank-avg on test
    def rank_norm_cols(mat):
        n = mat.shape[0]
        ranks = np.zeros_like(mat, dtype=np.float32)
        for j in range(mat.shape[1]):
            order = np.argsort(mat[:, j])
            ranks[order, j] = np.arange(n, dtype=np.float32)
        return ranks / (n - 1 + 1e-9)
    r_b4 = rank_norm_cols(logits_b4_test)
    r_b3 = rank_norm_cols(logits_b3_test)
    blend_rank = final_w * r_b4 + (1.0 - final_w) * r_b3
    probs = blend_rank  # already in [0,1]
else:
    blend_logits_test = final_w * logits_b4_test + (1.0 - final_w) * logits_b3_test
    probs = sigmoid_np(blend_logits_test)

sub = pd.DataFrame(probs, columns=target_cols)
sub.insert(0, id_col, ss_df[id_col].values)
out_path = 'submission.csv'
sub.to_csv(out_path, index=False)
print('[submit] Wrote', out_path, 'shape', sub.shape, '| method=', 'rank-avg' if use_rank else 'mean-logits', 'w=', round(final_w,2))

[oof] b4 OOF coverage: 27074 / 27074
[oof] fold 0 loading chkpts/effb3_fold0.pt @size 512


/tmp/ipykernel_1774/2806399247.py:42: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):


[oof] fold 1 loading chkpts/effb3_fold1.pt @size 512


[oof] fold 2 loading chkpts/effb3_fold2.pt @size 512


[oof] fold 3 loading chkpts/effb3_fold3.pt @size 512


...

[test] loading chkpts/effb3_fold1.pt @size 512


[test] loading chkpts/effb3_fold2.pt @size 512


[test] loading chkpts/effb3_fold3.pt @size 512


[test] loading chkpts/effb3_fold4.pt @size 512


[test] saved test logits b4/b3, time 843.0 s
[submit] Wrote submission.csv shape (3009, 10) | method= mean-logits w= 0.7


In [20]:
# Step 3: Finetune b4_ns at 768 on selected folds (multi-scale boost). Will save chkpts/effb4ns768_fold{f}.pt
import time, gc, os, numpy as np, torch, torch.nn as nn, torch.optim as optim
from timm.scheduler import CosineLRScheduler
import timm
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader

FINETUNE_FOLDS = [0, 2]  # best folds from 640 run
FT_IMG_SIZE = 768
FT_EPOCHS = 3
FT_BS = 4
FT_ACCUM = 4  # effective 16
FT_LR = 5e-5
FT_WD = 1e-4
FT_LS = 0.05

def evaluate_macro_auc_probs(y_true_np, probs_np):
    aucs = []
    for i in range(y_true_np.shape[1]):
        try:
            aucs.append(roc_auc_score(y_true_np[:, i], probs_np[:, i]))
        except Exception:
            pass
    return float(np.mean(aucs)) if len(aucs)>0 else 0.0, aucs

def ft_get_loaders(fold: int, size: int):
    train_idx = folds_df.index[folds_df['fold'] != fold].tolist()
    valid_idx = folds_df.index[folds_df['fold'] == fold].tolist()
    train_ds = RanzcrDataset(folds_df.iloc[train_idx], train_dir, id_col, target_cols, transforms=build_train_aug(size, clahe_p=1.0))
    valid_ds = RanzcrDataset(folds_df.iloc[valid_idx], train_dir, id_col, target_cols, transforms=build_valid_aug(size, clahe_p=1.0))
    train_loader = DataLoader(train_ds, batch_size=FT_BS, shuffle=True, num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2)
    valid_loader = DataLoader(valid_ds, batch_size=FT_BS, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2)
    return train_loader, valid_loader

def ft_build_model(num_classes: int):
    model = timm.create_model('tf_efficientnet_b4_ns', pretrained=True, num_classes=num_classes, in_chans=3, cache_dir=os.path.abspath('hf_cache'))
    return model

def ft_sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def finetune_fold(fold: int):
    print(f"\n[ft] Fold {fold} finetune @ {FT_IMG_SIZE}")
    gc.collect()
    if DEVICE=='cuda':
        torch.cuda.empty_cache()
    train_loader, valid_loader = ft_get_loaders(fold, FT_IMG_SIZE)
    model = ft_build_model(num_classes=len(target_cols)).to(DEVICE).to(memory_format=torch.channels_last)
    # load 640 checkpoint weights
    base_ckpt = f"chkpts/effb4ns_fold{fold}.pt"
    assert os.path.exists(base_ckpt), f"Missing base ckpt: {base_ckpt}"
    state = torch.load(base_ckpt, map_location='cpu')
    model.load_state_dict(state['model'], strict=True)
    pos_w = torch.from_numpy(np.load('pos_weight.npy')).to(DEVICE).clamp(min=1.0, max=5.0)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_w)
    optimizer = optim.AdamW(model.parameters(), lr=FT_LR, weight_decay=FT_WD)
    steps_per_epoch = max(1, len(train_loader) // FT_ACCUM)
    total_steps = FT_EPOCHS * steps_per_epoch
    warmup_steps = max(1, steps_per_epoch // 2)
    lr_sched = CosineLRScheduler(optimizer, t_initial=total_steps, lr_min=max(1e-6, FT_LR*0.05), warmup_t=warmup_steps, warmup_lr_init=FT_LR/10.0, warmup_prefix=True, t_in_epochs=False)
    scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=='cuda'))
    best_auc = -1.0
    best_path = f"chkpts/effb4ns768_fold{fold}.pt"
    global_step = 0
    t_fold = time.time()
    for epoch in range(FT_EPOCHS):
        model.train()
        t0 = time.time()
        run_loss = 0.0
        optimizer.zero_grad(set_to_none=True)
        for it, (xb, yb) in enumerate(train_loader):
            xb = xb.to(DEVICE, non_blocking=True).to(memory_format=torch.channels_last)
            yb = yb.to(DEVICE, non_blocking=True)
            yb_s = yb * (1.0 - 2.0 * FT_LS) + FT_LS
            with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
                logits = model(xb)
                loss = criterion(logits, yb_s)
            scaler.scale(loss / FT_ACCUM).backward()
            if (it+1) % FT_ACCUM == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad(set_to_none=True)
                lr_sched.step_update(global_step)
                global_step += 1
            run_loss += loss.item()
            if (it+1) % 50 == 0:
                cur_lr = optimizer.param_groups[0]['lr']
                print(f"[ft {fold}] ep {epoch+1}/{FT_EPOCHS} it {it+1}/{len(train_loader)} loss {run_loss/(it+1):.4f} lr {cur_lr:.2e} elapsed {time.time()-t0:.1f}s")
        # validate
        model.eval()
        v_logits, v_targets = [], []
        with torch.no_grad():
            for xb, yb in valid_loader:
                xb = xb.to(DEVICE, non_blocking=True).to(memory_format=torch.channels_last)
                logits = model(xb)
                v_logits.append(logits.detach().cpu().numpy())
                v_targets.append(yb.numpy())
        v_logits = np.concatenate(v_logits, axis=0)
        v_targets = np.concatenate(v_targets, axis=0)
        v_probs = ft_sigmoid(v_logits)
        v_macro, v_aucs = evaluate_macro_auc_probs(v_targets, v_probs)
        print(f"[ft {fold}] ep {epoch+1} val macro AUC: {v_macro:.5f}")
        if v_macro > best_auc:
            best_auc = v_macro
            torch.save({'model': model.state_dict(), 'auc': best_auc}, best_path)
            print(f"[ft {fold}] saved best to {best_path}")
        print(f"[ft {fold}] ep {epoch+1} train_loss {run_loss/max(1,len(train_loader)):.4f} time {time.time()-t0:.1f}s")
    print(f"[ft {fold}] best macro AUC: {best_auc:.5f} total {time.time()-t_fold:.1f}s")

for f in FINETUNE_FOLDS:
    finetune_fold(f)
print('[ft] Done selected folds. Next: infer multi-scale by averaging logits from effb4ns_fold*.pt and effb4ns768_fold*.pt per fold.')


[ft] Fold 0 finetune @ 768


/app/.pip-target/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/app/.pip-target/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(


/tmp/ipykernel_1774/3806705381.py:61: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=='cuda'))


/tmp/ipykernel_1774/3806705381.py:75: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):


[ft 0] ep 1/3 it 50/5448 loss 0.6829 lr 5.73e-06 elapsed 64.4s


...

[ft 2] ep 1/3 it 4150/5484 loss 0.6902 lr 4.92e-05 elapsed 1033.4s


[ft 2] ep 1/3 it 4200/5484 loss 0.6903 lr 4.91e-05 elapsed 1045.9s


[ft 2] ep 1/3 it 4250/5484 loss 0.6902 lr 4.90e-05 elapsed 1058.3s


[ft 2] ep 1/3 it 4300/5484 loss 0.6903 lr 4.90e-05 elapsed 1070.7s


KeyboardInterrupt: 

In [39]:
# Step 2 prep: 8x TTA (+/-5, +/-10, hflip combos) and multi-scale per-fold inference (b4ns 640/768), b4-only submission
import numpy as np, pandas as pd, os, time, torch, cv2
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

def build_tta_list(size: int):
    base = [
        A.CLAHE(clip_limit=2.0, tile_grid_size=(8,8), p=1.0),
        A.LongestMaxSize(max_size=size),
        A.PadIfNeeded(min_height=size, min_width=size, border_mode=cv2.BORDER_REFLECT),
    ]
    norm = [A.Normalize(mean=MEAN, std=STD), ToTensorV2()]
    tta = []
    # 1) original
    tta.append(A.Compose(base + [A.NoOp()] + norm))
    # 2) hflip
    tta.append(A.Compose(base + [A.HorizontalFlip(p=1.0)] + norm))
    # 3) rotate +5
    tta.append(A.Compose(base + [A.Affine(rotate=(5,5), translate_percent=(0,0), scale=(1.0,1.0), border_mode=cv2.BORDER_REFLECT, p=1.0)] + norm))
    # 4) rotate -5
    tta.append(A.Compose(base + [A.Affine(rotate=(-5,-5), translate_percent=(0,0), scale=(1.0,1.0), border_mode=cv2.BORDER_REFLECT, p=1.0)] + norm))
    # 5) rotate +10
    tta.append(A.Compose(base + [A.Affine(rotate=(10,10), translate_percent=(0,0), scale=(1.0,1.0), border_mode=cv2.BORDER_REFLECT, p=1.0)] + norm))
    # 6) rotate -10
    tta.append(A.Compose(base + [A.Affine(rotate=(-10,-10), translate_percent=(0,0), scale=(1.0,1.0), border_mode=cv2.BORDER_REFLECT, p=1.0)] + norm))
    # 7) hflip + rot +5
    tta.append(A.Compose(base + [A.HorizontalFlip(p=1.0), A.Affine(rotate=(5,5), translate_percent=(0,0), scale=(1.0,1.0), border_mode=cv2.BORDER_REFLECT, p=1.0)] + norm))
    # 8) hflip + rot -5
    tta.append(A.Compose(base + [A.HorizontalFlip(p=1.0), A.Affine(rotate=(-5,-5), translate_percent=(0,0), scale=(1.0,1.0), border_mode=cv2.BORDER_REFLECT, p=1.0)] + norm))
    return tta

def predict_logits_ckpt_with_tta(ckpt_path: str, size: int, batch_size: int = 16):
    tta_list = build_tta_list(size)
    test_df = ss_df[[id_col]].copy()
    all_tta_logits = []
    model = load_model_for_infer(ckpt_path, num_classes=len(target_cols)).to(DEVICE)
    model.eval()
    with torch.no_grad():
        for tta_idx, t_aug in enumerate(tta_list):
            test_ds = RanzcrTestDataset(test_df, test_dir, id_col, transforms=t_aug)
            test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
            t_logits = []
            for xb, _ids in test_loader:
                xb = xb.to(DEVICE, non_blocking=True)
                with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
                    logits = model(xb)
                t_logits.append(logits.detach().cpu().numpy())
            all_tta_logits.append(np.concatenate(t_logits, axis=0))
    return np.mean(all_tta_logits, axis=0)  # [N, C]

def predict_b4_multiscale_8xTTA(batch_size: int = 16):
    # Per-fold: average logits across available scales {640, 768}, then average across folds
    fold_logits = []
    for f in range(5):
        logits_scales = []
        ckpt640 = f"chkpts/effb4ns_fold{f}.pt"
        if os.path.exists(ckpt640):
            print(f"[ms-tta] fold {f} b4@640")
            logits_scales.append(predict_logits_ckpt_with_tta(ckpt640, size=640, batch_size=batch_size))
        ckpt768 = f"chkpts/effb4ns768_fold{f}.pt"
        if os.path.exists(ckpt768):
            print(f"[ms-tta] fold {f} b4@768")
            logits_scales.append(predict_logits_ckpt_with_tta(ckpt768, size=768, batch_size=batch_size))
        if len(logits_scales) == 0:
            print(f"[ms-tta] fold {f} no b4 checkpoints found; skipping")
            continue
        fold_logits.append(np.mean(logits_scales, axis=0))
    assert len(fold_logits) > 0, 'No b4 multiscale fold logits collected'
    return np.mean(fold_logits, axis=0)

def predict_b4_640_clahe4_2xTTA(batch_size: int = 16):
    # Build 2x TTA with CLAHE clip_limit=4.0 for size=640
    test_df = ss_df[[id_col]].copy()
    base = [
        A.CLAHE(clip_limit=4.0, tile_grid_size=(8,8), p=1.0),
        A.LongestMaxSize(max_size=640),
        A.PadIfNeeded(min_height=640, min_width=640, border_mode=cv2.BORDER_REFLECT),
    ]
    norm = [A.Normalize(mean=MEAN, std=STD), ToTensorV2()]
    ttas = [
        A.Compose(base + [A.NoOp()] + norm),
        A.Compose(base + [A.HorizontalFlip(p=1.0)] + norm),
    ]
    fold_logits = []
    for f in range(5):
        ckpt = f"chkpts/effb4ns_fold{f}.pt"
        if not os.path.exists(ckpt):
            print(f"[clahe4] missing {ckpt}, skip")
            continue
        print(f"[clahe4] fold {f} b4@640 clahe4 2xTTA")
        model = load_model_for_infer(ckpt, num_classes=len(target_cols)).to(DEVICE)
        model.eval()
        tta_logits_fold = []
        with torch.no_grad():
            for t_aug in ttas:
                test_ds = RanzcrTestDataset(test_df, test_dir, id_col, transforms=t_aug)
                test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
                buf = []
                for xb, _ids in test_loader:
                    xb = xb.to(DEVICE, non_blocking=True)
                    with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
                        logits = model(xb)
                    buf.append(logits.detach().cpu().numpy())
                tta_logits_fold.append(np.concatenate(buf, axis=0))
        fold_logits.append(np.mean(tta_logits_fold, axis=0))
    assert len(fold_logits) > 0, 'No fold logits collected for clahe4'
    return np.mean(fold_logits, axis=0)

def predict_b4_640_reflect101_2xTTA(batch_size: int = 16):
    # Build 2x TTA with CLAHE clip_limit=2.0 and border_mode=REFLECT_101 for size=640
    test_df = ss_df[[id_col]].copy()
    base = [
        A.CLAHE(clip_limit=2.0, tile_grid_size=(8,8), p=1.0),
        A.LongestMaxSize(max_size=640),
        A.PadIfNeeded(min_height=640, min_width=640, border_mode=cv2.BORDER_REFLECT_101),
    ]
    norm = [A.Normalize(mean=MEAN, std=STD), ToTensorV2()]
    ttas = [
        A.Compose(base + [A.NoOp()] + norm),
        A.Compose(base + [A.HorizontalFlip(p=1.0)] + norm),
    ]
    fold_logits = []
    for f in range(5):
        ckpt = f"chkpts/effb4ns_fold{f}.pt"
        if not os.path.exists(ckpt):
            print(f"[ref101] missing {ckpt}, skip")
            continue
        print(f"[ref101] fold {f} b4@640 REFLECT_101 2xTTA")
        model = load_model_for_infer(ckpt, num_classes=len(target_cols)).to(DEVICE)
        model.eval()
        tta_logits_fold = []
        with torch.no_grad():
            for t_aug in ttas:
                test_ds = RanzcrTestDataset(test_df, test_dir, id_col, transforms=t_aug)
                test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
                buf = []
                for xb, _ids in test_loader:
                    xb = xb.to(DEVICE, non_blocking=True)
                    with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
                        logits = model(xb)
                    buf.append(logits.detach().cpu().numpy())
                tta_logits_fold.append(np.concatenate(buf, axis=0))
        fold_logits.append(np.mean(tta_logits_fold, axis=0))
    assert len(fold_logits) > 0, 'No fold logits collected for reflect101'
    return np.mean(fold_logits, axis=0)

def sigmoid_np(x):
    return 1.0 / (1.0 + np.exp(-x))

def run_b4_ms_8xTTA_submission(batch_size: int = 12):
    if DEVICE=='cuda':
        torch.cuda.empty_cache()
    t0 = time.time()
    logits_b4_ms = predict_b4_multiscale_8xTTA(batch_size=batch_size)
    np.save('test_logits_b4_ms_8tta.npy', logits_b4_ms)
    probs = sigmoid_np(logits_b4_ms)
    sub = pd.DataFrame(probs, columns=target_cols)
    sub.insert(0, id_col, ss_df[id_col].values)
    sub.to_csv('submission_b4_ms_8tta.csv', index=False)
    print(f"[ms-tta] wrote submission_b4_ms_8tta.csv shape {sub.shape} time {round(time.time()-t0,1)}s")

def run_b4_clahe4_2tta_and_ensemble(batch_size: int = 12):
    # Run 640@clahe4 2xTTA, then average logits with existing b4_ms_8tta if present; otherwise just write clahe4 probs
    if DEVICE=='cuda':
        torch.cuda.empty_cache()
    t0 = time.time()
    logits_clahe4 = predict_b4_640_clahe4_2xTTA(batch_size=batch_size)
    np.save('test_logits_b4_640_clahe4_2tta.npy', logits_clahe4)
    if os.path.exists('test_logits_b4_ms_8tta.npy'):
        logits_ms = np.load('test_logits_b4_ms_8tta.npy')
        blend_logits = 0.8 * logits_ms + 0.2 * logits_clahe4
        probs = sigmoid_np(blend_logits)
        out = 'submission_b4_ms8tta_plus_clahe4.csv'
        sub = pd.DataFrame(probs, columns=target_cols)
        sub.insert(0, id_col, ss_df[id_col].values)
        sub.to_csv(out, index=False)
        print(f"[clahe4] wrote {out} shape {sub.shape} time {round(time.time()-t0,1)}s")
    else:
        probs = sigmoid_np(logits_clahe4)
        out = 'submission_b4_640_clahe4_2tta.csv'
        sub = pd.DataFrame(probs, columns=target_cols)
        sub.insert(0, id_col, ss_df[id_col].values)
        sub.to_csv(out, index=False)
        print(f"[clahe4] wrote {out} shape {sub.shape} time {round(time.time()-t0,1)}s")

def run_b4_reflect101_2tta_and_ensemble(batch_size: int = 12):
    # Run 640@REFLECT_101 2xTTA, then average logits with b4_ms_8tta if present
    if DEVICE=='cuda':
        torch.cuda.empty_cache()
    t0 = time.time()
    logits_r101 = predict_b4_640_reflect101_2xTTA(batch_size=batch_size)
    np.save('test_logits_b4_640_reflect101_2tta.npy', logits_r101)
    if os.path.exists('test_logits_b4_ms_8tta.npy'):
        logits_ms = np.load('test_logits_b4_ms_8tta.npy')
        blend_logits = 0.85 * logits_ms + 0.15 * logits_r101
        probs = sigmoid_np(blend_logits)
        out = 'submission_b4_ms8tta_plus_reflect101.csv'
        sub = pd.DataFrame(probs, columns=target_cols)
        sub.insert(0, id_col, ss_df[id_col].values)
        sub.to_csv(out, index=False)
        print(f"[ref101] wrote {out} shape {sub.shape} time {round(time.time()-t0,1)}s")
    else:
        probs = sigmoid_np(logits_r101)
        out = 'submission_b4_640_reflect101_2tta.csv'
        sub = pd.DataFrame(probs, columns=target_cols)
        sub.insert(0, id_col, ss_df[id_col].values)
        sub.to_csv(out, index=False)
        print(f"[ref101] wrote {out} shape {sub.shape} time {round(time.time()-t0,1)}s")

# After this cell: call run_b4_ms_8xTTA_submission(batch_size=10) in Cell 10; optional: run_b4_clahe4_2tta_and_ensemble(batch_size=12) to blend; or run_b4_reflect101_2tta_and_ensemble(batch_size=12)

In [ ]:
# Execute b4 640@REFLECT_101 2xTTA and ensemble with existing b4_ms_8tta logits; write final submission.csv
import os
batch = 12
run_b4_reflect101_2tta_and_ensemble(batch_size=batch)
out1 = 'submission_b4_ms8tta_plus_reflect101.csv'
out2 = 'submission_b4_640_reflect101_2tta.csv'
final_out = out1 if os.path.exists(out1) else out2
os.replace(final_out, 'submission.csv')
print('[final] submission.csv overwritten from', final_out, 'batch=', batch)

[ref101] fold 0 b4@640 REFLECT_101 2xTTA


/app/.pip-target/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(


/tmp/ipykernel_1774/2360268179.py:140: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):


In [23]:
# Quick CPU-only reblend from saved logits: b4-only, per-class b4/b3 weights, and global w=0.9
import numpy as np, pandas as pd, os, time
from sklearn.metrics import roc_auc_score

def sigmoid_np(x):
    return 1.0 / (1.0 + np.exp(-x))

# Load saved test logits
b4_640_path = 'test_logits_b4_640.npy'  # 5-fold b4@640 with hflip TTA
b3_512_path = 'test_logits_b3_512.npy'  # 5-fold b3@512 with hflip TTA
assert os.path.exists(b4_640_path), f"Missing {b4_640_path}; run Cell 7 to generate"
logits_b4_test = np.load(b4_640_path)
logits_b3_test = np.load(b3_512_path) if os.path.exists(b3_512_path) else None
print('[reblend] Loaded test logits shapes:', logits_b4_test.shape, None if logits_b3_test is None else logits_b3_test.shape)

# Write b4-only submission
probs_b4_only = sigmoid_np(logits_b4_test)
sub_b4 = pd.DataFrame(probs_b4_only, columns=target_cols)
sub_b4.insert(0, id_col, ss_df[id_col].values)
sub_b4.to_csv('submission_b4only_640.csv', index=False)
print('[reblend] wrote submission_b4only_640.csv')

# If b3 logits exist, compute per-class weights using OOF and generate blended submissions
if logits_b3_test is not None and os.path.exists('oof_logits_all.npy') and os.path.exists('oof_logits_b3.npy'):
    oof_b4 = np.load('oof_logits_all.npy')  # shape [N, C]
    oof_b3 = np.load('oof_logits_b3.npy')   # shape [N, C]
    mask_b4 = ~np.isnan(oof_b4).any(axis=1)
    mask_b3 = ~np.isnan(oof_b3).any(axis=1)
    mask = mask_b4 & mask_b3
    y_true = folds_df.loc[mask, target_cols].values.astype(np.float32)
    o4 = oof_b4[mask]
    o3 = oof_b3[mask]
    print('[reblend] OOF covered:', mask.sum(), '/', len(mask))

    # Per-class weight search: maximize per-class AUC over w in [0,1]
    ws = np.linspace(0.0, 1.0, 21)
    best_w_per_class = np.zeros(o4.shape[1], dtype=np.float32)
    per_class_auc = []
    for j in range(o4.shape[1]):
        yj = y_true[:, j]
        best_auc = -1.0
        best_w = 1.0
        for w in ws:
            z = w * o4[:, j] + (1.0 - w) * o3[:, j]
            try:
                auc = roc_auc_score(yj, sigmoid_np(z))
            except Exception:
                continue
            if auc > best_auc:
                best_auc = auc
                best_w = float(w)
        best_w_per_class[j] = best_w
        per_class_auc.append(best_auc)
    print('[reblend] per-class best weights:', np.round(best_w_per_class, 2).tolist())

    # Apply per-class weights to test logits
    W = best_w_per_class.reshape(1, -1)
    blend_pc_logits = W * logits_b4_test + (1.0 - W) * logits_b3_test
    probs_pc = sigmoid_np(blend_pc_logits)
    sub_pc = pd.DataFrame(probs_pc, columns=target_cols)
    sub_pc.insert(0, id_col, ss_df[id_col].values)
    sub_pc.to_csv('submission_blend_perclass.csv', index=False)
    print('[reblend] wrote submission_blend_perclass.csv')

    # Global heavier b4 weight w=0.9
    w = 0.9
    blend_w09 = w * logits_b4_test + (1.0 - w) * logits_b3_test
    probs_w09 = sigmoid_np(blend_w09)
    sub_w09 = pd.DataFrame(probs_w09, columns=target_cols)
    sub_w09.insert(0, id_col, ss_df[id_col].values)
    sub_w09.to_csv('submission_blend_w09.csv', index=False)
    print('[reblend] wrote submission_blend_w09.csv')

    # Choose per-class blend as default final submission
    os.replace('submission_blend_perclass.csv', 'submission.csv')
    print('[reblend] Final submission.csv set to per-class blended b4/b3')
else:
    # Fallback to b4-only as final submission
    os.replace('submission_b4only_640.csv', 'submission.csv')
    print('[reblend] Final submission.csv set to b4-only 640')

[reblend] Loaded test logits shapes: (3009, 9) (3009, 9)
[reblend] wrote submission_b4only_640.csv
[reblend] OOF covered: 27074 / 27074


[reblend] per-class best weights: [1.0, 0.20000000298023224, 0.4000000059604645, 0.25, 0.3499999940395355, 0.20000000298023224, 0.3499999940395355, 0.550000011920929, 0.550000011920929]
[reblend] wrote submission_blend_perclass.csv
[reblend] wrote submission_blend_w09.csv
[reblend] Final submission.csv set to per-class blended b4/b3


In [27]:
# Fast CPU blend: b4_ms_8tta (strong) + b4_640 (weak) + b3_4tta (very light), then submit
import numpy as np, pandas as pd, os

def sigmoid_np(x):
    return 1.0 / (1.0 + np.exp(-x))

paths = {
    'b4_ms': 'test_logits_b4_ms_8tta.npy',
    'b4_640': 'test_logits_b4_640.npy',
    'b3_4tta': 'test_logits_b3_4tta.npy',
}
for k,p in paths.items():
    if not os.path.exists(p):
        print(f"[blend] missing {k}: {p}")

logits_b4_ms = np.load(paths['b4_ms']) if os.path.exists(paths['b4_ms']) else None
logits_b4_640 = np.load(paths['b4_640']) if os.path.exists(paths['b4_640']) else None
logits_b3_4tta = np.load(paths['b3_4tta']) if os.path.exists(paths['b3_4tta']) else None

# Heavily favor b4_ms; lightly mix others if present
blend = None
w_ms, w_b4, w_b3 = 0.85, 0.10, 0.05
if logits_b4_ms is None:
    raise FileNotFoundError('test_logits_b4_ms_8tta.npy not found')
blend = w_ms * logits_b4_ms
if logits_b4_640 is not None:
    blend = blend + w_b4 * logits_b4_640
if logits_b3_4tta is not None:
    blend = blend + w_b3 * logits_b3_4tta

probs = sigmoid_np(blend)
sub = pd.DataFrame(probs, columns=target_cols)
sub.insert(0, id_col, ss_df[id_col].values)
sub_path = 'submission_blend_b4ms_b4_b3.csv'
sub.to_csv(sub_path, index=False)
os.replace(sub_path, 'submission.csv')
print('[blend] Wrote submission.csv from weighted blend (b4_ms 0.85, b4_640 0.10, b3 0.05). Shapes:',
      None if logits_b4_ms is None else logits_b4_ms.shape,
      None if logits_b4_640 is None else logits_b4_640.shape,
      None if logits_b3_4tta is None else logits_b3_4tta.shape)

[blend] Wrote submission.csv from weighted blend (b4_ms 0.85, b4_640 0.10, b3 0.05). Shapes: (3009, 9) (3009, 9) (3009, 9)


In [28]:
# Rank-average blend of available test logits (b4_ms_8tta, b4_640, b3_4tta) with heavy b4_ms weight
import numpy as np, pandas as pd, os

def rank_normalize_cols(mat: np.ndarray) -> np.ndarray:
    n, c = mat.shape
    ranks = np.zeros_like(mat, dtype=np.float32)
    for j in range(c):
        order = np.argsort(mat[:, j])
        ranks[order, j] = np.arange(n, dtype=np.float32)
    return ranks / (n - 1 + 1e-9)

paths = {
    'b4_ms': 'test_logits_b4_ms_8tta.npy',
    'b4_640': 'test_logits_b4_640.npy',
    'b3_4tta': 'test_logits_b3_4tta.npy',
}
logits = {}
for k, p in paths.items():
    if os.path.exists(p):
        logits[k] = np.load(p)
    else:
        print(f"[rank-blend] missing {k}: {p}")

assert 'b4_ms' in logits, 'Missing b4_ms logits'
r_blend = rank_normalize_cols(logits['b4_ms']) * 0.9
if 'b4_640' in logits:
    r_blend += rank_normalize_cols(logits['b4_640']) * 0.08
if 'b3_4tta' in logits:
    r_blend += rank_normalize_cols(logits['b3_4tta']) * 0.02

# r_blend is already in [0,1]
sub = pd.DataFrame(r_blend, columns=target_cols)
sub.insert(0, id_col, ss_df[id_col].values)
sub.to_csv('submission_rank_blend.csv', index=False)
os.replace('submission_rank_blend.csv', 'submission.csv')
print('[rank-blend] Wrote submission.csv (rank-avg) shapes:',
      None if 'b4_ms' not in logits else logits['b4_ms'].shape,
      None if 'b4_640' not in logits else logits['b4_640'].shape,
      None if 'b3_4tta' not in logits else logits['b3_4tta'].shape)

[rank-blend] Wrote submission.csv (rank-avg) shapes: (3009, 9) (3009, 9) (3009, 9)


In [32]:
# Write submission from CLAHE4-only 640 2xTTA logits and set as final
import numpy as np, pandas as pd, os
path = 'test_logits_b4_640_clahe4_2tta.npy'
assert os.path.exists(path), f"Missing {path}; run Cell 10 CLAHE4 step first"
logits = np.load(path)
probs = 1.0 / (1.0 + np.exp(-logits))
sub = pd.DataFrame(probs, columns=target_cols)
sub.insert(0, id_col, ss_df[id_col].values)
sub.to_csv('submission_b4_640_clahe4_only.csv', index=False)
os.replace('submission_b4_640_clahe4_only.csv', 'submission.csv')
print('[final-alt] submission.csv written from CLAHE4-only 640 2xTTA logits')

[final-alt] submission.csv written from CLAHE4-only 640 2xTTA logits


In [33]:
# Per-class OOF-tuned blend: apply weights learned on (b4@640 vs b3@512) to test (b4_ms_8tta vs b3_4tta)
import numpy as np, pandas as pd, os
from sklearn.metrics import roc_auc_score

def sigmoid_np(x):
    return 1.0 / (1.0 + np.exp(-x))

# Load OOF logits for weight tuning (b4@640 and b3@512)
oof_b4_path = 'oof_logits_all.npy'
oof_b3_path = 'oof_logits_b3.npy'
assert os.path.exists(oof_b4_path) and os.path.exists(oof_b3_path), 'Missing OOF logits files'
o4 = np.load(oof_b4_path)
o3 = np.load(oof_b3_path)
mask = (~np.isnan(o4).any(axis=1)) & (~np.isnan(o3).any(axis=1))
y_true = folds_df.loc[mask, target_cols].values.astype(np.float32)
o4m = o4[mask]
o3m = o3[mask]

# Line search per class for w in [0,1]
ws = np.linspace(0.0, 1.0, 21)
best_w = np.ones(o4m.shape[1], dtype=np.float32)
for j in range(o4m.shape[1]):
    yj = y_true[:, j]
    best_auc, bw = -1.0, 1.0
    for w in ws:
        z = w * o4m[:, j] + (1.0 - w) * o3m[:, j]
        try:
            auc = roc_auc_score(yj, sigmoid_np(z))
        except Exception:
            continue
        if auc > best_auc:
            best_auc, bw = auc, float(w)
    best_w[j] = bw
print('[pc-blend] per-class best w from OOF (b4@640 vs b3@512):', np.round(best_w, 2).tolist())

# Load test logits for b4_ms_8tta (strong) and b3_4tta (diversity); apply per-class weights
b4_ms_path = 'test_logits_b4_ms_8tta.npy'
b3_4t_path = 'test_logits_b3_4tta.npy'
assert os.path.exists(b4_ms_path), 'Missing test logits: b4_ms_8tta'
logits_ms = np.load(b4_ms_path)
logits_b3 = np.load(b3_4t_path) if os.path.exists(b3_4t_path) else None
if logits_b3 is None:
    print('[pc-blend] Missing b3 test logits; falling back to b4_ms only')
    probs = sigmoid_np(logits_ms)
else:
    W = best_w.reshape(1, -1)
    logits_blend = W * logits_ms + (1.0 - W) * logits_b3
    probs = sigmoid_np(logits_blend)

sub = pd.DataFrame(probs, columns=target_cols)
sub.insert(0, id_col, ss_df[id_col].values)
sub.to_csv('submission_b4ms_b3_perclass.csv', index=False)
os.replace('submission_b4ms_b3_perclass.csv', 'submission.csv')
print('[pc-blend] Wrote submission.csv using per-class OOF-tuned weights between b4_ms_8tta and b3_4tta')

[pc-blend] per-class best w from OOF (b4@640 vs b3@512): [1.0, 0.20000000298023224, 0.4000000059604645, 0.25, 0.3499999940395355, 0.20000000298023224, 0.3499999940395355, 0.550000011920929, 0.550000011920929]
[pc-blend] Wrote submission.csv using per-class OOF-tuned weights between b4_ms_8tta and b3_4tta


In [35]:
# Set submission.csv to strongest available candidate by priority
import os, shutil
candidates = [
    'submission_b4_ms8tta_plus_clahe4.csv',
    'submission_b4_ms_8tta.csv',
    'submission_blend_b4ms_b4_b3.csv',
    'submission_rank_blend.csv',
    'submission_b4only_640.csv'
]
chosen = None
for p in candidates:
    if os.path.exists(p):
        chosen = p
        break
assert chosen is not None, 'No candidate submission file found'
shutil.copy(chosen, 'submission.csv')
print('[final-pick] submission.csv overwritten from', chosen)

[final-pick] submission.csv overwritten from submission_b4only_640.csv


In [36]:
# Recreate b4_ms8tta_plus_clahe4 submission from saved logits and set as final
import numpy as np, pandas as pd, os
def sigmoid_np(x):
    return 1.0 / (1.0 + np.exp(-x))
ms_path = 'test_logits_b4_ms_8tta.npy'
clahe4_path = 'test_logits_b4_640_clahe4_2tta.npy'
assert os.path.exists(ms_path), 'Missing test_logits_b4_ms_8tta.npy'
assert os.path.exists(clahe4_path), 'Missing test_logits_b4_640_clahe4_2tta.npy'
logits_ms = np.load(ms_path)
logits_c4 = np.load(clahe4_path)
blend_logits = 0.8 * logits_ms + 0.2 * logits_c4
probs = sigmoid_np(blend_logits)
sub = pd.DataFrame(probs, columns=target_cols)
sub.insert(0, id_col, ss_df[id_col].values)
out_path = 'submission_b4_ms8tta_plus_clahe4.csv'
sub.to_csv(out_path, index=False)
os.replace(out_path, 'submission.csv')
print('[recreate] submission.csv written from 0.8*ms8tta + 0.2*clahe4')

[recreate] submission.csv written from 0.8*ms8tta + 0.2*clahe4
